In [ ]:
import requests
import lxml
import pandas as pd
import IPython
import shutil
from tqdm import tqdm
from bs4 import BeautifulSoup as Soup
from datetime import datetime , timedelta
from IPython.display import Image

In [ ]:
l=[]

for i in tqdm(range(35)):
    i+=1
    url='https://www.theguardian.com/type/gallery/'+(datetime.today()-timedelta(i)).strftime('%Y/%b/%d')+'/all' 
    r=requests.get(url)
    html = Soup(r.text, 'lxml') 
    l.extend(list(set([i['href'] for i in html.find_all('a') if i.get('href') is not None and '/gallery/' in i['href'] and not i['href'].endswith(('/all', '/altdate'))])))
    i+=1
    del r
    

In [ ]:
l2=[]
for p in tqdm(l):
    r=requests.get(p)
    category=p.split('https://www.theguardian.com/')[1].split('/')[0]
    html = Soup(r.text, 'lxml')
    z=html.find_all("picture")
    l2.extend([[i.findChildren()[1]['srcset'].split(' ')[0],p,category] for i in z if i.parent['class'][0]=='u-responsive-ratio' ])

In [ ]:
df=pd.DataFrame(l2)
df.columns = ['image_url', 'page_url', 'category']

In [ ]:
from IPython.display import Image
r=df.sample(1)
# r=df[df['category']=='world'].sample(1)

print (r.category.item())
print (r.page_url.item())
Image(url= r.image_url.item())
# Image(url= r.image_url.item(), width=200, height=200)

In [ ]:
r=df.sample(1)
url=r.image_url.item()
response=requests.get(url, stream=True)
with open('img.jpg', 'wb') as out_file:
    shutil.copyfileobj(response.raw, out_file)
del response

In [ ]:
df[df['category']=='world']['page_url'].value_counts()